# Helper function for programmatic pages
This resource helps to generate a couple of landing page around a data source. This resource can help with programmatic SEO

This notebook:
 - Loads a csv datasource from **data/**
 - collects templates from **templates/**
 - Writes an overview page linking to the subpages in **output/**
 - Writes several subpages in **output/**
 - Writes the additions to a sitemap in **output/**

In [34]:
# Here we read the data source for our programmatic pages
import pandas as pd
df = pd.read_csv("data/languages.csv")
lang_variation_dict = df.groupby(['Language']).apply(lambda x: x['Variation'].tolist()).to_dict()

## Helper functions

In [1]:
# takes a html template file marked up with elements to replace
# returns the content as string
def open_template(path):
    text_file = open(path, "r")
    template = text_file.read()
    text_file.close()
    return template

# takes a string with html contents
# writes it to file
def write_page(path, str):
    f = open(path, "a")
    f.write(str)
    f.close()   
    return

def insert_values_into_html_template(html_string, replacement_dict):
    for replacement in replacement_dict.keys():
        html_string = html_string.replace(replacement, replacement_dict[replacement])
    return html_string

def generate_html_link_list(list_of_elements, link_base="https://example.com/"):
    return "".join(["<li><a href='%s%s.html'>%s</a></li>" % (link_base, x.lower(), x) for x in list_of_elements])

def amount_phrase(x, terms = ("%s item", "%s items")):
    phrase = ""
    if x == 1:
        phrase = terms[0] % (x)
    else: 
        phrase = terms[1] % (x)
    return phrase


## Generating the overview
First we generate the overview page that links to the specific subpages

In [13]:
# We load the template for the page that links to the programmatic pages
overview_template = open_template("templates/multilingual.html")

# We construct a HTML fragment with a series of links in list elements
html_fragment = generate_html_link_list(lang_variation_dict.keys(), "https://stenos.io/")

# We construct a dict of replacements
# And we generate the overview page by replacing the bits in the template
replacement_dict = {"{list_of_languages}":html_fragment}   
overview_page = insert_values_into_html_template(overview_template, replacement_dict)

#And we write it to file
write_page("output/multilingual.html", overview_page)

## Generating the specific subpages

In [43]:
subpage_template = open_template("templates/language.html")

for x in lang_variation_dict.keys():
    
    # We construct the fragments we want to sub in
    language = x
    num_variants = amount_phrase(len(lang_variation_dict[x]), ["% variant", "%s variants"]
    list_of_variants = ", ".join(lang_variation_dict[x])
    
    # We construct a dict of replacements
    # And we generate the overview page by replacing the bits in the template
    replacement_dict = {
        "{list_of_variants}":list_of_variants,
        "{language}":language,
        "{num_variants}":num_variants
    }   
    subpage = insert_values_into_html_template(subpage_template, replacement_dict)
    write_page("output/%s.html"%(x.lower()), subpage)

Afrikaans (South Africa)
Amharic (Ethiopia)
Arabic (Algeria), Arabic (Bahrain), modern standard, Arabic (Egypt), Arabic (Iraq), Arabic (Israel), Arabic (Jordan), Arabic (Kuwait), Arabic (Lebanon), Arabic (Libya), Arabic (Morocco), Arabic (Oman), Arabic (Palestinian Authority), Arabic (Qatar), Arabic (Saudi Arabia), Arabic (Syria), Arabic (Tunisia), Arabic (United Arab Emirates), Arabic (Yemen)
Bulgarian (Bulgaria)
Burmese (Myanmar)
Catalan (Spain)
Chinese (Cantonese, Traditional), Chinese (Mandarin, Simplified), Chinese (Taiwanese Mandarin)
Croatian (Croatia)
Czech (Czech)
Danish (Denmark)
Dutch (Belgium), Dutch (Netherlands)
English (Australia), English (Canada), English (Ghana), English (Hong Kong), English (India), English (Ireland), English (Kenya), English (New Zealand), English (Nigeria), English (Philippines), English (Singapore), English (South Africa), English (Tanzania), English (United Kingdom), English (United States)
Estonian (Estonia)
Filipino (Philippines)
Finnish (Finla

## Finally update the sitemap

In [12]:
sitemap_template = open_template("templates/sitemap.xml")

# We construct the fragment we want to sub in
pages_fragment = ""
for x in lang_variation_dict.keys():
    pages_fragment += """<url>
        <loc>https://example.com/{0}.html</loc>
        <lastmod>2022-05-05</lastmod>
        <xhtml:link rel='alternate' hreflang='en' href='https://example.com/{0}.html'/>
    </url>""".format(x.lower())

# We construct a dict of replacements
# And we generate the overview page by replacing the bits in the template
replacement_dict = {"{pages}":pages_fragment}   
sitemap = insert_values_into_html_template(sitemap_template, replacement_dict)
write_page("output/sitemap.xml", sitemap)